# 😃 Evaluate OCR API with Specific Test Dataset

## 🥸 Metrics for Evaluate

In [6]:
import editdistance

ground_truth = "นายนนท์ปวิธ เป็นนักศึกษา"
predicted_text = "นายนนปวิธ เป็น"

### 🥹 Character Error Rate

In [7]:
total_chars = sum(len(gt) for gt in ground_truth)
cer = sum(editdistance.eval(gt, pt) for gt, pt in zip(ground_truth, predicted_text)) / total_chars
print(f"Character Error Rate (CER): {cer * 100:.2f}%")

Character Error Rate (CER): 37.50%


### 😉 Word Error Rate

In [8]:
ground_words = [gt.split() for gt in ground_truth]
predicted_words = [pt.split() for pt in predicted_text]
total_words = sum(len(words) for words in ground_words)
wer = sum(editdistance.eval(gt, pt) for gt, pt in zip(ground_words, predicted_words)) / total_words
print(f"Word Error Rate (WER): {wer * 100:.2f}%")

Word Error Rate (WER): 39.13%


In [9]:
from editdistance import eval as edit_distance
def calculate_wer(predicted_text, ground_truth_text):
    # Split texts into lists of words
    predicted_words = predicted_text.split()
    ground_truth_words = ground_truth_text.split()
    
    # Calculate edit distance between word lists
    distance = edit_distance(predicted_words, ground_truth_words)
    
    # Calculate WER
    wer = distance / len(ground_truth_words)
    return wer

## 😈 OCR API from Google Vision API


In [10]:
# Setting key
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../keys/subtle-bus-443807-m7-801043491063.json"

In [11]:
from google.cloud import vision
import io

def detect_text(image_path):
    # Create client for Google Vision
    client = vision.ImageAnnotatorClient()
    # Load image
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    # Calling OCR function
    response = client.text_detection(image=image)
    texts = response.text_annotations
    if response.error.message:
        raise Exception(f"API Error: {response.error.message}")
    return texts[0].description 

# Calling detect_text()
predicted_text = detect_text("../../assets/output/angsa_1_augmented.png")
print(predicted_text.replace('\n', ' '))

ชื่อ รัฐพงษ์ บินดี บัตรประจำตัว 1588486511756 REG014 วันผลิต 2024-09-29 วันหมดอายุ 2026-09-05 วันเกิด 20 มีนาคม 2524 คาร์บามาซีพีน 200mg เม็ดอมใต้ลิ้น ใช้ทาในเวลากลางคืนก่อนนอน บรรเทาอาการอักเสบ หากมีอาการปวดศีรษะรุนแรง ควรหยุดยา


In [12]:
with open("../../assets/output/angsa_1.txt", encoding="utf-8") as file:
        ground_truth_text = file.read()
print(ground_truth_text.replace('\n', ' '))

ชื่อ รัฐพงษ์ บินดี บัตรประจำตัว 1588486511756 วันเกิด 20 มีนาคม 2524 REG014 วันผลิต 2024-09-29 วันหมดอายุ 2026-09-05 คาร์บามาซีพีน 200mg เม็ดอมใต้ลิ้น ใช้ทาในเวลากลางคืนก่อนนอน บรรเทาอาการอักเสบ หากมีอาการปวดศีรษะรุนแรง ควรหยุดยา 


In [13]:
# Function to calculate Character Error Rate
def calculate_cer(predicted_text, ground_truth_text):
    return editdistance.eval(predicted_text, ground_truth_text) / len(ground_truth_text)

cer = calculate_cer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
wer = calculate_wer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
print(f"CER: {cer:.2%} | WER: {wer:.2%}")

CER: 20.43% | WER: 38.10%


In [14]:
sum_character_error_rate = 0
sum_word_error_rate = 0

for i in range(1,51):
    predicted_text = detect_text(f"../../assets/output/angsa_{i}_augmented.png")
    with open(f"../../assets/output/angsa_{i}.txt", encoding="utf-8") as file:
        ground_truth_text = file.read()
    cer = calculate_cer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
    wer = calculate_wer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
    sum_character_error_rate += cer
    sum_word_error_rate += wer
average_character_error_rate = sum_character_error_rate/10
average_sum_word_error_rate = sum_word_error_rate/10
print(f"Average CER: {cer:.2%} | Average WER: {wer:.2%}")

Average CER: 14.40% | Average WER: 16.00%


## 😈 OCR API from PyTesseract


In [15]:
import pytesseract
from PIL import Image

def ocr_image(image_path, lang='tha+eng'):
    # Load Image
    image = Image.open(image_path)
    # Calling OCR function
    text = pytesseract.image_to_string(image, lang=lang)
    return text

In [47]:
print(ocr_image("../../assets/output/angsa_1_augmented.png").replace('\n', ' '))

REGO13  


In [46]:
with open("../../assets/output/angsa_1.txt", encoding="utf-8") as file:
        ground_truth_text = file.read()
print(ground_truth_text.replace('\n',' '))

ชื่อ ณาร์รีมาน เตมิยะเดช บัตรประจำตัว 7493515973128 วันเกิด 13 กุมภาพันธ์ 2525 REG013 วันผลิต 2024-05-25 วันหมดอายุ 2026-01-23 CLARITHROMYCIN 250mg TABLET รับประทานวันละ 1 เม็ดหลังมื้อกลางวัน ลดน้ำตาลในเลือด ควรใช้ยาในปริมาณที่ถูกต้อง 


In [16]:
sum_character_error_rate = 0
sum_word_error_rate = 0
for i in range(1,51):
    print(i)
    predicted_text = ocr_image(f"../../assets/output/angsa_{i}_augmented.png")
    with open(f"../../assets/output/angsa_{i}.txt", encoding="utf-8") as file:
        ground_truth_text = file.read()
    cer = calculate_cer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
    wer = calculate_wer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
    sum_character_error_rate += cer
    sum_word_error_rate += wer

average_character_error_rate = sum_character_error_rate/10
average_sum_word_error_rate = sum_word_error_rate/10
print(f"Average CER: {cer:.2%} | Average WER: {wer:.2%}")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Average CER: 36.21% | Average WER: 44.00%
